Данный скрипт позволяет 

In [ ]:
import numpy as np
import math
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
from numpy import linalg as LA

In [32]:
#Все маркеры
markers = ['RAH', 'RPH', 'LAH', 'LPH', 'C7', 'T8', 'IJ', 'PX','RSC', 'RAC','LSC', 'LAC', 'RAI', 'RPC',
           'RTS', 'RAA', 'LAI', 'LTS', 'LAA', 'LPC', 'REM', 'REL', 'LEM', 'LEL', 'RRS', 'RUS', 'LRS', 'LUS', 'LSJ', 'RSJ']
#Хранит названия колонок в списке
cols = []
#Маркеры, условно неподвижные относительно тела
stablemarkers = ['T8', 'IJ', 'PX', 'C7']
#Колонки, соответствующие условно неподвижным маркерам
stablecols = []
#Хранит названия колонок в строке
csvcols = ''
for i in markers:
    cols.append(i + 'x')
    cols.append(i + 'y')
    cols.append(i + 'z')
    csvcols += cols[-3] + '\t' + cols[-2] + '\t' + cols[-1] + '\t'
for i in stablemarkers:
    stablecols.append(i + 'x')
    stablecols.append(i + 'y')
    stablecols.append(i + 'z')

In [30]:
#Файлы для анализа
files = ['1.trc', '2.trc','3.trc','4.trc','5.trc','6.trc','7.trc','8.trc','9.trc','9_combined.trc','9_left.trc','9_right.trc',
'10.trc', '11.trc','12.trc','12_shrugging.trc','13.trc','14.trc','14_depression.trc', '14_elevation.trc', 'very long static 14.trc']
#files = ['1.trc']
#Добавление префикса/постфикса
for i in range(len(files)):
    files[i] = 'smoothed_trc\\' + files[i][0:-4] + '_shifted_smoothed.trc'
for file in files:
    #Data from original file
    datalines = []
    with open (file, 'r') as fp:
        datalines = fp.readlines()
        infolines = datalines[0:5]
        datalines = datalines[5:-1]
    textdatalines = ''
    textinfodatalines = ''
    for i in datalines:
        textdatalines += i + '\n'
    #df хранит данные оригинального файла
    df = pd.read_csv(StringIO(textdatalines), sep = '\t', names = ['Frame#', 'Time'] + cols)
    
    #Still Subject coordinates in laboratory frame. Describe the movement of the point, which is still relatively the body
    SSx = np.zeros(len(df.index))
    SSy = np.zeros(len(df.index))
    SSz = np.zeros(len(df.index))

    for i in stablecols:
        if 'x' in i:
            SSx += np.array(df[i])
        elif 'y' in i:
            SSy += np.array(df[i])
        elif 'z' in i:
            SSz += np.array(df[i])

    SSx /= len(stablecols) 
    SSy /= len(stablecols)
    SSz /= len(stablecols)
    
    #critical std differentiates still markers between moving ones. when the std is greater than 3sigma for a relatively
    #still point, a marker must be recognised as a moving one. sigma is calculated as 6 maximal std of still subject's
    #coordinates
    critstd = 6 * max(SSx.std(), SSy.std(), SSz.std())
    
    cols_sorted = cols.copy()
    #Сортируем маркеры по величине стандартного отклонения
    cols_sorted = sorted(cols_sorted, key = lambda x: df[x].std())
    
    #Количество маркеров с наиболее крупными стандартными отклонениями
    numbiggest = math.ceil(0.15 * len(df.columns))
    #Суммарное std маркеров с наиболее крупными стандартными отклонениями
    stdleaders = 0
    for i in range(1, numbiggest + 1):
        stdleaders += df[cols_sorted[-i]].std()
    stdleaders /= numbiggest
    
    with open (file[0:-4] + '_markersinfo.txt', 'w+') as fp:
        boundary_not_written = True
        fp.write('<Marker> <std, mm> <suggested weight>\n')
        for i in cols_sorted:
            curstd = df[i].std()
            if boundary_not_written and (curstd > critstd):
                fp.write('=== Here start relatively moving markers ===\n')
                boundary_not_written = False
            if curstd < critstd / 2:
                fp.write(i + ' %.2f -- 1.0\n' %(curstd))
            else:
                fp.write(i + ' %.2f -- %.1f\n' %(curstd, max(1, 5 * np.sqrt(curstd / stdleaders))))


KeyboardInterrupt: 